# Limpieza inicial

Este notebook documenta el proceso de razonamiento para limpiar la data proporcionada. Al final se genera un script con la data limpia para ser preprocesada.

## Cargar los datos que usaremos

Usaremos los archivos entrenamiento_precios_vivienda.csv  y prueba_precios_vivienda.csv. Note que los datos del archivo prueba_precios_vivienda.csv no contienen la columna de los precios de la vivienda. La idea de este archivo, es que usted complete dicha columna con los predicciones resultantes de su modelo, y mediante un proceso de validación externo, Learning Code calcula el desempeño de este. Esta es una práctica muy común en pruebas de este tipo.

In [1]:
import pandas as pd
from content.utils.data_processing import load_csv_data, set_index

trainData = load_csv_data("./content/sample_data/train.csv")
testData = load_csv_data("./content/sample_data/test.csv")
# Ahora asignamos un index para manejar de manera mas eficiente la data.
indexedTrainData = set_index(trainData)
indexedTestData = set_index(testData)

print(f'Train data have {indexedTrainData.shape[0]} rows, \nTest data only {indexedTestData.shape[0]}')

Train data have 9629 rows, 
Test data only 3175


Ahora veamos que data esta vacia en cada columna

In [2]:
indexedTrainData[indexedTrainData.columns[indexedTrainData.isnull().any()]].isnull().sum()

fecha_aprobación                        5610
tipo_subsidio                           8489
municipio_inmueble                         1
barrio                                     5
sector                                     1
direccion_inmueble_informe                 7
descripcion_tipo_inmueble                176
descripcion_uso_inmueble                 183
descripcion_clase_inmueble               177
area_actividad                             1
tipo_vigilancia                            2
observaciones_estructura                   1
observaciones_dependencias                 1
numero_garaje_1                           21
numero_garaje_2                            3
matricula_garaje_5                         1
matricula_inmobiliaria_deposito_3          1
matricula_inmobiliaria_deposito_4          1
observaciones_generales_construccion      17
valor_uvr                                  1
Longitud                                  29
Latitud                                  127
dtype: int

Pocos datos faltantes, pero los pocos que no estan pueden rellenarse facilmente. El tipo de subsidio tiene demasiados datos faltantes.

In [3]:
indexedTestData[indexedTestData.columns[indexedTestData.isnull().any()]].isnull().sum()

fecha_aprobación                        1892
tipo_subsidio                           2831
barrio                                     2
descripcion_tipo_inmueble                 61
descripcion_uso_inmueble                  62
descripcion_clase_inmueble                61
numero_garaje_1                            7
numero_garaje_2                            1
matricula_inmobiliaria_deposito_3          1
observaciones_generales_construccion       3
valor_area_privada                      3174
area_garaje                                1
valor_area_garaje                       3174
area_deposito                              1
valor_area_deposito                     3174
area_terreno                               1
valor_area_terreno                      3174
area_construccion                          1
valor_area_construccion                 3174
area_otros                                 1
valor_area_otros                        3174
area_libre                                 1
valor_area

Aparte de los datos faltantes en el valor (entendible pues son datos que deben rellenarse), se nota que el tipo de subsidio tiene demasiados datos faltantes. En los demas pueden rellenarse los datos. Fecha de aprobacion y tipo de subsidio de nuevo tienen muchos datos faltantes.

## Limpiando los datos

Una columna que tiene demasiados datos faltantes en ambos dataframes es el de fecha de aprobación, por lo que se removera. Tambien podemos alegar que la fecha no deberia ser relevante, por lo menos en comparacion a otros parametros.

Tipo de subsidio tambien vemos que presenta muchos datos nulos, se eliminira.

### Data que no aporta nada

Las columnas que consideramos que tienen poca informacion relevante y deben borrarse son:

In [4]:
extra_data = [
  'fecha_aprobación',
  'tipo_subsidio',
  'numero_garaje_1',
  'matricula_garaje_1',
  'numero_garaje_2',
  'matricula_garaje_2',
  'numero_garaje_3',
  'matricula_garaje_3',
  'numero_garaje_4',
  'matricula_garaje_4',
  'numero_garaje_5',
  'matricula_garaje_5',
  'numero_deposito_1',
  'matricula_inmobiliaria_deposito_1',
  'numero_deposito_2',
  'matricula_inmobiliaria_deposito_2',
  'numero_deposito_3',
  'matricula_inmobiliaria_deposito_3',
  'numero_deposito_4',
  'matricula_inmobiliaria_deposito_4',
  'numero_deposito_5',
  'matricula_inmobiliaria_deposito_5',
  'metodo_valuacion_1',
  'concepto_del_metodo_1',
  'metodo_valuacion_2',
  'concepto_del_metodo_2',
  'metodo_valuacion_3',
  'concepto_del_metodo_3',
  'metodo_valuacion_4',
  'concepto_del_metodo_4',
  'metodo_valuacion_5',
  'concepto_del_metodo_5',
  'metodo_valuacion_6',
  'concepto_del_metodo_6',
  'metodo_valuacion_7',
  'concepto_del_metodo_7',
  'metodo_valuacion_8',
  'concepto_del_metodo_8',
  'metodo_valuacion_9',
  'concepto_del_metodo_9',
  'Longitud',
  'Latitud',
  'tipo_deposito',
  'numero_total_depositos',
]

Se proporciono el archivo `PuntosInteres.csv`, creemos que podria comparase con las columnas `Longitud` y `Latitud`, pero requiere clasificar los tipos de interes, por ejemplo "sera que la farmacia es beneficiosa para el valuo?". Por lo tanto decidimos que no se tomara en cuenta dada la limitante de tiempo.

### Valorizacion en base a la percepcion y la descripcion del inmueble

Por otro lado tenemos algunas que consideramos que pueden arrojan informacion pero necesitan trabajo adicional, mas que todo preprocesar intensamente la data, para extraerle un score o sentimiento:

Estas se trabajaran aparte y se integraran despues a la data para el entrenamiento.

In [5]:
descriptions_related = [
  'descripcion_clase_inmueble',
  'perspectivas_de_valorizacion',
  'actualidad_edificadora',
  'comportamiento_oferta_demanda',
  'observaciones_generales_inmueble',
  'observaciones_estructura',
  'observaciones_generales_construccion',
  'observaciones_dependencias',
  'descripcion_tipo_inmueble',
  'descripcion_uso_inmueble',
]

### Influencia de la zona

Un caso interesante aca es el de 'sector' pues usualmente rural siempre vale menos que urbano por metro cuadrado. Sin embargo combinandolo con los anteriores puede dar mas informacion sobre la zona especifica, por ejemplo inidicar que un apartamento en la zona urbana tiene mayor valor que un lote urbano.

Pero las 3 ultimas son las que dan mas informacion, pues indican con palabras si el barrio o la ciudad estan cotizadas. Creemos que usar tambien departamento, municipio o barrio implicaria en la practica hacer una etiquetacion de tales columnas (es bogota bueno, malo o no afecta al valor?). Por eso se incluyo al final a las 3 primeras para eliminar, pues no aportan tanta data como se quisiera.

In [6]:
zone_related = [
  'departamento_inmueble',
  'municipio_inmueble',
  'barrio',
  'descripcion_general_sector',
  'direccion_inmueble_informe',
  'descripcion_general_sector',
]

### Influencia de la estructura

Se refieren a elementos de la infraestructura en si, por ejemplo si la estructura se ve suficientemente segura.

Se sacara por el momento y se integrara luego de haberse analizado aparte.

In [7]:
structure_related = [
  'observaciones_generales_inmueble',
  'observaciones_estructura',
  'observaciones_dependencias',
  'observaciones_generales_construccion',
]

### Area, altura, dimensiones

Aca podria pensarse que el area total y el area contruida serian los valores mas importantes, y otros valores no ayudan individualmente, asi mismo la atura no se toma en cuenta.

In [8]:
dimensions_related = [
  'area_privada',
  'area_garaje',
  'area_deposito',
  'area_terreno',
  'area_construccion',
  'area_otros',
  'area_libre',
]

Notamos que hay una seccion de "garages", segun se analiza la columna que podria aportar mas informacion es la `numero_total_de_garajes` y alternativamente `total_cupos_parquedaro`, las demas columnas son redundantes.

In [9]:
garage_related = [
  'garaje_cubierto_1',
  'garaje_doble_1',
  'garaje_paralelo_1',
  'garaje_servidumbre_1',
  'garaje_cubierto_2',
  'garaje_doble_2',
  'garaje_paralelo_2',
  'garaje_servidumbre_2',
  'garaje_cubierto_3',
  'garaje_doble_3',
  'garaje_paralelo_3',
  'garaje_servidumbre_3',
  'garaje_cubierto_4',
  'garaje_doble_4',
  'garaje_paralelo_4',
  'garaje_servidumbre_4',
  'garaje_cubierto_5',
  'garaje_doble_5',
  'garaje_paralelo_5',
  'garaje_servidumbre_5',
  'garaje_visitantes', # ya oncluido en el numero de garages
]

Otra seccion es la referente a las normas de contruccion, pensamos que no son importantes

In [10]:
norms_solumns = [
  'altura_permitida',
  'observaciones_altura_permitida', 
  'aislamiento_posterior',
  'observaciones_aislamiento_posterior', 
  'aislamiento_lateral',
  'observaciones_aislamiento_lateral', 
  'antejardin',
  'observaciones_antejardin', 
  'indice_ocupacion', 
  'observaciones_indice_ocupacion', 
  'indice_construccion',
  'observaciones_indice_construccion', 
  'predio_subdividido_fisicamente', # Si | No (Contiene datos espureos)
  'rph', # (Muchas, se nota tambien muchos datos espureos)
  'sometido_a_propiedad_horizontal', # Si | No (Contiene datos espureos)
]

### Columnas de casos muy especificos

Se refiere a columnas que solo aplican a casos no muy generales, por ejemplo el de numero de unidades se refiere a cuantas subdivisiones tiene un predio, pero si notamos no puede aplicarse a la mayoria de casos que son casas o terrenos individuales, en cuyo casos se asigna usualmente un cero.

In [11]:
specific_cases = [
  'condicion_ph',
  'ajustes_sismoresistentes', # No Disponibles | No Reparados | Reparados (Contiene datos espureos)
  'danos_previos', # No disponible | Sin daños previos | Con daños previos (Contiene datos espureos)
]

### Valor

Aca solo tomamos el valor total, las demas quedan borradas

Nos quedaremos con `valor_total_avaluo` pues el que se requiere en la descripcion del proyecto, aunque 'valor_avaluo_en_uvr' es independiente de la fecha, es decir que muestra mas claro la diferencia de comprar un terreno en el 2000 contra comprar en el 2020, por ejemplo.

In [12]:
value_related = [
  'valor_area_privada',
  'valor_area_garaje',
  'valor_area_deposito',
  'valor_area_terreno',
  'valor_area_construccion',
  'valor_area_otros',
  'valor_area_libre',
  'valor_uvr',
  'valor_avaluo_en_uvr',
]

Con lo anterior podemos limpiar la data inicial de columnas innecesarias, y luego integrar las que se analizaran aparte.

In [13]:
columnsToErase = extra_data + descriptions_related + zone_related + structure_related + dimensions_related + garage_related + norms_solumns + value_related + specific_cases

cleanTrainData = indexedTrainData.drop(columnsToErase, axis=1)
cleanTestData = indexedTestData.drop(columnsToErase, axis=1)

print(f'Train data now have {cleanTrainData.shape}')
print(f'Test data now have {cleanTestData.shape}')

Train data now have (9629, 106)
Test data now have (3175, 106)


In [14]:
cleanTrainData.head()

,objeto,motivo,proposito,tipo_avaluo,tipo_credito,sector,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,...,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,valor_total_avaluo
id,,,,,,,,,,,,,,,,,,,,,
1,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,Urbano,Si,Si,Si,Si,...,Bueno,Normal,Bueno,Normal,Bueno,Integral,No Tiene,0,0,71995500
5,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,Urbano,Si,Si,Si,Si,...,Bueno,Normal,Bueno,Normal,Bueno,Semi-Integral,No Tiene,0,0,78090000
7,Originación,Empleados,Garantía Hipotecaria,Hipotecario,Vivienda,Urbano,Si,Si,Si,Si,...,Bueno,Normal,Bueno,Normal,Bueno,Integral,Privado,1,1,1648560000516
10,Originación,Crédito hipotecario de vivienda,Garantía Hipotecaria,Hipotecario,Vivienda,Urbano,Si,Si,Si,Si,...,Bueno,Sencillo,Regular,Sencillo,Regular,Sencillo,No Tiene,0,0,96610000
11,Originación,Leasing Visto Bueno,Garantía Hipotecaria,Hipotecario,Vivienda,Urbano,Si,Si,Si,Si,...,Bueno,Normal,Bueno,Normal,Bueno,Integral,Privado,3,4,1608208000


Seleccionemos primero las columnas con data categorica

In [15]:
categorical_columns = [
  # Seccion avaluo
  'objeto', # Originación | Remate (Contiene datos espureos)
  'motivo', # Crédito hipotecario de vivienda | Empleados | Leasing Visto Bueno | Leasing Habitacional | Remates | Garantía | Actualizacion de garantias | Colomext Hipotecario | Credito Comercial | Compra de cartera | Dacion en Pago | Leasing Comercial | Reformas | Originacion | Leasing Inmobiliario - Persona Natural
  'proposito', # Crédito hipotecario de vivienda | Garantía Hipotecaria | Transaccion Comercial de Venta | Valor Asegurable
  'tipo_avaluo', # Hipotecario | Remates | Garantia Hipotecaria
  'tipo_credito', # Vivienda | Diferente de Vivienda | Hipotecario
  # Seccion Informacion general y situacional
  'sector', # Urbano | Rural | Poblado (Contiene datos espureos)
  # Seccion Informacion del inmueble
  'tipo_inmueble', # Apartamento | Casa | Casa Rural | Conjunto o Edificio | Deposito | Finca | Garaje | Lote | Lote Urbano | Oficina (Contiene datos espureos)
  'uso_actual', # (Muchas, se nota tambien muchos datos espureos)
  'clase_inmueble', # (Muchas, se nota tambien muchos datos espureos)
  'ocupante', # (Muchas, se nota tambien muchos datos espureos)
  'area_actividad', # (Muchas, se nota tambien muchos datos espureos)
  'uso_principal_ph', # Vivienda | Finca | Viviend, Serv y Comercio (Muchas, se nota tambien muchos datos espureos)
  'estructura', # Mamposteria Estructural | Tradicional | Industrializada | Muro de carga (Contiene datos espureos)
  'cubierta', # Teja Metalica | Teja Plastica | Tradicional | Teja fibrocemento | Teja de Barro (Contiene datos espureos)
  'fachada', # Concreto texturado | Flotante | Graniplast | Industrilizada | Ladrillo a la vista (Contiene datos espureos)
  'estructura_reforzada', # Flotante | Graniplast | Trabes coladas en sitio | No tiene trabes (Contiene datos espureos)
  'material_de_construccion', # Acero | Adobe, bahareque o tapia | Concreto Reforzado (Contiene datos espureos)
  'detalle_material', # Mampostería reforzada | Pórticos | Mampostería confinada (Contiene datos espureos)
  'iluminacion', # Bueno | Paneles prefabricados | Muros (Contiene datos espureos)
  'calidad_acabados_cocina', # Integral | Semi-Integral | Sencillo | Bueno | Lujoso | Normal | Regular | Sin Acabados
  # Seccion Garage
  'tipo_garaje', # Bueno | Comunal | Exclusivo | Integral | Lujoso | No tiene | Normal | Privado | Regular | Semi-Integral | Sencillo | Sin Acabados
]

In [16]:
binary_columns = [
  'alcantarillado_en_el_sector', # Si | No (Contiene datos espureos)
  'acueducto_en_el_sector', # Si | No (Contiene datos espureos)
  'gas_en_el_sector', # Si | No
  'energia_en_el_sector', # Si | No
  'telefono_en_el_sector', # Si | No
  'vias_pavimentadas', # Si | No
  'sardineles_en_las_vias', # Si | No
  'andenes_en_las_vias', # Si | No
  'barrio_legal', # Si | No (Contiene datos espureos)
  'paradero', # Si | No (Contiene datos espureos)
  'alumbrado', # Si | No (Contiene datos espureos)
  'arborizacion', # Si | No (Contiene datos espureos)
  'alamedas', # Si | No
  'ciclo_rutas', # Si | No
  'alcantarillado_en_el_predio', # Si | No (Contiene datos espureos)
  'acueducto_en_el_predio', # Si | No (Contiene datos espureos)
  'gas_en_el_predio', # Si | No (Contiene datos espureos)
  'energia_en_el_predio', # Si | No (Contiene datos espureos)
  'telefono_en_el_predio', # Si | No (Contiene datos espureos)
  'porteria', # Si | No (Contiene datos espureos)
  'citofono', # Si | No (Contiene datos espureos)
  'bicicletero', # Si | No (Contiene datos espureos)
  'piscina', # Si | No (Contiene datos espureos)
  'tanque_de_agua', # Si | No (Contiene datos espureos)
  'club_house', # Si | No (Contiene dato espureo "0", podria tomarse como No)
  'teatrino', # Si | No (Contiene dato espureo "0", podria tomarse como No)
  'sauna', # Si | No (Contiene dato espureo "0", podria tomarse como No)
  'vigilancia_privada', # Si | No (Contiene dato espureo "0", podria tomarse como No)
  'administracion', # Si | No (Contiene datos espureos)
]

### Datos ordinales

Expresan una cualidad a través de un dato que es posible ordenar a través de una escala previamente definida.

In [17]:
ordinal_columns = [
  'estrato', # 1 - 6 (Contiene datos espureos)
  'topografia_sector', # Inclinado | Ligera | Plano (Contiene datos espureos)
  'condiciones_salubridad', # Buenas | Malas | Regulares (Contiene datos espureos)
  'transporte', # Bueno | Regular | Malo (Contiene datos espureos)
  'demanda_interes', # Nula | Bueno | Debil | Fuerte (Contiene datos espureos)
  'nivel_equipamiento_comercial', # En Proyecto | Regular Malo | Bueno | Muy bueno (Contiene datos espureos)
  'tipo_vigilancia', # 12 Horas | 24 Horas | No (Dato espureo Si podria tomarse como "24 Horas", dato espureo "0" podria tomarse como "No")
  'tipo_fachada', # De 0 a 3 metros | de 3 a 6 metros | Mayor a 6 metros (Contiene datos espureos)
  'ventilacion', # Bueno | Regular | Malo (Contiene datos espureos)
  'irregularidad_planta', # Sin irregularidad | No disponible | Con irregularidad (Contiene datos espureos)
  'irregularidad_altura', # Sin irregularidad | No disponible | Con irregularidad (Contiene datos espureos)=
  'estado_acabados_cocina', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
  'estado_acabados_pisos', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'calidad_acabados_pisos', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'estado_acabados_muros', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'calidad_acabados_muros', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'estado_acabados_techos', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'calidad_acabados_techos', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'estado_acabados_madera', # Bueno | Sin Acabados | Normal | Sencillo (Contiene datos espureos)
  'calidad_acabados_madera', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
  'estado_acabados_metal', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
  'calidad_acabados_metal', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
  'estado_acabados_banos', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
  'calidad_acabados_banos', # Bueno | Lujoso | Malo | Normal | Regular | Sencillo | Sin acabados
]

### Datos numericos

Estos datos son expresados en números y sí que pueden medirse.

In [18]:
numeric_columns = [
  # Seccion Informacion del inmueble
  'unidades', # [Int] 0 - 92 (Contiene datos espureos)
  'contadores_agua', # [Int] 0 - 6 (Contiene datos espureos "92", "Aplica", "No", podria asumirse que es cero, "Resultante")
  'contadores_luz', # [Int] 0 - 6 (Contiene datos espureos "92", "Aplica", "No", podria asumirse que es cero)
  'accesorios', # # [Int] 0 - 46 (Contiene dato espureo "No", podria asumirse que es cero)
  'area_valorada', # [Float] 0.0 - 1058.2 (Contiene unos numeros gitantes)
  'numero_piso', # [Int] 0 - 99 (Contiene datos espureos)
  'numero_de_edificios', # [Int] 0 - 99 (Contiene datos espureos)
  'vetustez', # a veces dice anhos antiguedad, a veces el anho de construccion
  'pisos_bodega', # [Int] 0 - 52 (Contiene datos espureos)
  'habitaciones', # [Int] 0 - 32 (Contiene datos espureos)
  'estar_habitacion', # [Int] 0 - 9 (Contiene datos espureos)
  'cuarto_servicio', # [Int] 0 - 5 (Contiene datos espureos)
  'closet', # [Int] 0 - 17 (Contiene datos espureos)
  'sala', # [Int] 0 - 24 (Contiene datos espureos)
  'comedor', # [Int] 0 - 31 (Contiene datos espureos)
  'bano_privado', # [Int] 0 - 24 (Contiene datos espureos)
  'bano_social', # [Int] 0 - 12
  'bano_servicio', # [Int] 0 - 11
  'cocina', # [Int] 0 - 13
  'estudio', # [Int] 0 - 3
  'balcon', # [Int] 0 - 11
  'terraza', # [Int] 0 - 9
  'patio_interior', # [Int] 0 - 11
  'jardin', # [Int] 0 - 4
  'zona_de_ropas', # [Int] 0 - 13
  'zona_verde_privada', # [Int] 0 - 4
  'local', # [Int] 0 - 10
  'oficina', # [Int] 0 - 9
  'bodega', # [Int] 0 - 2
  # Seccion Garage
  'numero_total_de_garajes', # [Int] 0 - 5 (Contiene datos espureos)
  'total_cupos_parquedaro', # [Int] 0 - 8 (Contiene datos espureos)
]

Se notan varios datos espureos, en el id 13365 pudimos notar que las columnas estan movidas, se movio la data manualmente.

Otro arreglo que se realizo sobre la data 3204
3730
11042
11601
entre otros que tenia columnas movidas 

Se eliminaron

472
485
547
621
833
837
1419
1574
1675
1749
336
540
595
728
773
958
1473
1532
2746
2855
4008
4365
4669
4783
5243
5269
5986
6216
6414
7200
7420
7934
8124
8390
8814
10047
10543
11321
12305
12839
16048
17239
119
280
394
629
843
931
1292
1394
1722
1777
2997


Al inspeccionar visualmente la data, la mayoria por que el valor total esta vacio, junto a otros errores
13983
417
526
596
730
1594
2403
2769
2871
3568
3645
3835
3864
3911
4031
4059
4135
4168
4267
4492
4653
4726
4983
5001
5496
5935
6372
6866
6919
7106
7529
7586
7871
7982
8089
9602
9618
10289
10753
10883
11184
11810
12396
12530
13195
13204
13420
14108
14210
14225
14689
15230
16057
16204
16508
16540
16764
16850
16924
17057
17234
17236
17442
17543
17686
17728
17796
17806
18143
18214
2670
4612
6403


Con respecto al documento de pruebas tambien se hallaron problemas que se solucionaron de la siguiente manera

Se modifico varios que tenian el mismo error que en el de entrenamiento.


Por ultimo, se procedio a terminar de limpiar data eliminando celdas vacias.

El el caso de las columnas numericas se relleno con ceros cuando la celda esta vacia.

In [19]:
cleanTrainData[numeric_columns] = cleanTrainData[numeric_columns].fillna(0)
for column in numeric_columns:
  cleanTrainData[column] = cleanTrainData[column].str.replace(",", ".")

cleanTrainData[numeric_columns] = cleanTrainData.loc[:,numeric_columns].transform(lambda x: x.map(lambda x: { "Si": 1., "No": 0. }.get(x,x)))

cleanTrainData[numeric_columns] = cleanTrainData[numeric_columns].apply(pd.to_numeric).astype(float)
cleanTrainData[numeric_columns].isnull().sum()

unidades                   0
contadores_agua            0
contadores_luz             0
accesorios                 0
area_valorada              0
numero_piso                0
numero_de_edificios        0
vetustez                   0
pisos_bodega               0
habitaciones               0
estar_habitacion           0
cuarto_servicio            0
closet                     0
sala                       0
comedor                    0
bano_privado               0
bano_social                0
bano_servicio              0
cocina                     0
estudio                    0
balcon                     0
terraza                    0
patio_interior             0
jardin                     0
zona_de_ropas              0
zona_verde_privada         0
local                      0
oficina                    0
bodega                     0
numero_total_de_garajes    0
total_cupos_parquedaro     0
dtype: int64

In [20]:
cleanTrainData[numeric_columns]

,unidades,contadores_agua,contadores_luz,accesorios,area_valorada,numero_piso,numero_de_edificios,vetustez,pisos_bodega,habitaciones,...,terraza,patio_interior,jardin,zona_de_ropas,zona_verde_privada,local,oficina,bodega,numero_total_de_garajes,total_cupos_parquedaro
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,53.33,4.0,99.0,6.0,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,57.60,0.0,0.0,7.0,1.0,3.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,62.44,9.0,7.0,9.0,2.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
10,0.0,0.0,0.0,0.0,128.80,0.0,0.0,35.0,2.0,4.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,153.69,5.0,1.0,7.0,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18327,0.0,0.0,0.0,0.0,54.40,5.0,2.0,2.0,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
18328,0.0,0.0,0.0,0.0,41.09,25.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
18329,0.0,0.0,0.0,0.0,96.60,0.0,0.0,48.0,2.0,5.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


Ahora las columnas booleanas convertirlas a numeros (1/0)

In [21]:
cleanTrainData[binary_columns] = cleanTrainData.loc[:,binary_columns].transform(lambda x: x.map(lambda x: { "Si": 1., "No": 0. }.get(x,x)))
cleanTrainData[binary_columns] = cleanTrainData[binary_columns].fillna(0.).apply(pd.to_numeric).astype(float)
cleanTrainData[binary_columns].isnull().sum()

alcantarillado_en_el_sector    0
acueducto_en_el_sector         0
gas_en_el_sector               0
energia_en_el_sector           0
telefono_en_el_sector          0
vias_pavimentadas              0
sardineles_en_las_vias         0
andenes_en_las_vias            0
barrio_legal                   0
paradero                       0
alumbrado                      0
arborizacion                   0
alamedas                       0
ciclo_rutas                    0
alcantarillado_en_el_predio    0
acueducto_en_el_predio         0
gas_en_el_predio               0
energia_en_el_predio           0
telefono_en_el_predio          0
porteria                       0
citofono                       0
bicicletero                    0
piscina                        0
tanque_de_agua                 0
club_house                     0
teatrino                       0
sauna                          0
vigilancia_privada             0
administracion                 0
dtype: int64

In [22]:
cleanTrainData[binary_columns]

,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,barrio_legal,paradero,...,porteria,citofono,bicicletero,piscina,tanque_de_agua,club_house,teatrino,sauna,vigilancia_privada,administracion
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18327,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
18328,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
18329,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora trataremos individualmente algunas columnas ordinales

In [23]:
cleanTrainData['estrato'] = cleanTrainData.loc[:,'estrato'].transform(lambda x: x.map(lambda x: { "Comercial": 7., "Oficina": 8., "Industrial": 9., "No": 0. }.get(x,x)))

cleanTrainData['topografia_sector'] = cleanTrainData.loc[:,'topografia_sector'].transform(lambda x: x.map(lambda x: { "Plano": 0., "Ligera": 1., "Inclinado": 2., "Accidentada": 3., "No": 0. }.get(x,x)))

cleanTrainData['condiciones_salubridad'] = cleanTrainData.loc[:,'condiciones_salubridad'].transform(lambda x: x.map(lambda x: { "Malas": 0., "Bueno": 1., "Buenas": 1., "Regulares": 2., "Malas": 3., "No": 0. }.get(x,x)))

cleanTrainData['transporte'] = cleanTrainData.loc[:,'transporte'].transform(lambda x: x.map(lambda x: { "Malo": 0., "Regular": 1., "Bueno": 2., "Vivienda": 3., "Hotelero": 4., "No": 0. }.get(x,x)))

cleanTrainData['demanda_interes'] = cleanTrainData.loc[:,'demanda_interes'].transform(lambda x: x.map(lambda x: { "Nula": 0., "Débil": 1., "Media": 2., "Bueno": 3., "Fuerte": 4., "No": 0. }.get(x,x)))

cleanTrainData['nivel_equipamiento_comercial'] = cleanTrainData.loc[:,'nivel_equipamiento_comercial'].transform(lambda x: x.map(lambda x: { "En Proyecto": 1., "Regular Malo": 0., "Bueno": 2., "Muy bueno": 3., "No": 0. }.get(x,x)))

cleanTrainData['tipo_vigilancia'] = cleanTrainData.loc[:,'tipo_vigilancia'].transform(lambda x: x.map(lambda x: { "12 Horas": 1., "24 Horas": 2., "No": 0. }.get(x,x)))

cleanTrainData['tipo_fachada'] = cleanTrainData.loc[:,'tipo_fachada'].transform(lambda x: x.map(lambda x: { "De 0 a 3 metros": 1., "De 3 a 6 metros": 2., "Mayor a 6 metros": 3., "No": 0. }.get(x,x)))

cleanTrainData['ventilacion'] = cleanTrainData.loc[:,'ventilacion'].transform(lambda x: x.map(lambda x: { "Malo": 0., "Regular": 1., "Bueno": 2., "No": 0. }.get(x,x)))

cleanTrainData['irregularidad_planta'] = cleanTrainData.loc[:,'irregularidad_planta'].transform(lambda x: x.map(lambda x: { "No disponible": 0., "Con irregularidad": 1., "Sin irregularidad": 2., "No": 0. }.get(x,x)))

cleanTrainData['irregularidad_altura'] = cleanTrainData.loc[:,'irregularidad_altura'].transform(lambda x: x.map(lambda x: { "No disponible": 0., "Con irregularidad": 1., "Sin irregularidad": 2., "No": 0. }.get(x,x)))

dictionary_details = { "Malo": 0., "Sin Acabados": 1., "Sin acabados": 1., "Sencillo": 2., "Normal": 4., "Bueno": 5., "Lujoso": 5., "No disponible": 0., "Regular": 3., "No": 0.}

cleanTrainData['estado_acabados_cocina'] = cleanTrainData.loc[:,'estado_acabados_cocina'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_pisos'] = cleanTrainData.loc[:,'estado_acabados_pisos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_pisos'] = cleanTrainData.loc[:,'calidad_acabados_pisos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_muros'] = cleanTrainData.loc[:,'estado_acabados_muros'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_muros'] = cleanTrainData.loc[:,'calidad_acabados_muros'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_techos'] = cleanTrainData.loc[:,'estado_acabados_techos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_techos'] = cleanTrainData.loc[:,'calidad_acabados_techos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_madera'] = cleanTrainData.loc[:,'estado_acabados_madera'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_madera'] = cleanTrainData.loc[:,'calidad_acabados_madera'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_metal'] = cleanTrainData.loc[:,'estado_acabados_metal'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_metal'] = cleanTrainData.loc[:,'calidad_acabados_metal'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['estado_acabados_banos'] = cleanTrainData.loc[:,'estado_acabados_banos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))

cleanTrainData['calidad_acabados_banos'] = cleanTrainData.loc[:,'calidad_acabados_banos'].transform(lambda x: x.map(lambda x: dictionary_details.get(x,x)))


cleanTrainData[ordinal_columns] = cleanTrainData[ordinal_columns].fillna(0.).apply(pd.to_numeric).astype(float)
cleanTrainData[ordinal_columns].isnull().sum()


estrato                         0
topografia_sector               0
condiciones_salubridad          0
transporte                      0
demanda_interes                 0
nivel_equipamiento_comercial    0
tipo_vigilancia                 0
tipo_fachada                    0
ventilacion                     0
irregularidad_planta            0
irregularidad_altura            0
estado_acabados_cocina          0
estado_acabados_pisos           0
calidad_acabados_pisos          0
estado_acabados_muros           0
calidad_acabados_muros          0
estado_acabados_techos          0
calidad_acabados_techos         0
estado_acabados_madera          0
calidad_acabados_madera         0
estado_acabados_metal           0
calidad_acabados_metal          0
estado_acabados_banos           0
calidad_acabados_banos          0
dtype: int64

In [24]:
cleanTrainData[ordinal_columns]

,estrato,topografia_sector,condiciones_salubridad,transporte,demanda_interes,nivel_equipamiento_comercial,tipo_vigilancia,tipo_fachada,ventilacion,irregularidad_planta,...,estado_acabados_muros,calidad_acabados_muros,estado_acabados_techos,calidad_acabados_techos,estado_acabados_madera,calidad_acabados_madera,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos
id,,,,,,,,,,,,,,,,,,,,,
1,2.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
5,2.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
7,3.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
10,3.0,0.0,1.0,2.0,2.0,2.0,0.0,1.0,2.0,2.0,...,5.0,4.0,5.0,4.0,3.0,2.0,5.0,2.0,3.0,2.0
11,6.0,1.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0,0.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18327,3.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,0.0,...,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,4.0
18328,3.0,0.0,1.0,2.0,2.0,3.0,2.0,3.0,2.0,0.0,...,5.0,2.0,5.0,2.0,5.0,4.0,5.0,4.0,5.0,4.0
18329,3.0,1.0,1.0,2.0,2.0,3.0,0.0,3.0,2.0,0.0,...,5.0,2.0,5.0,2.0,5.0,2.0,5.0,2.0,5.0,2.0


Por ultimo convertimos las categoricas en dummies

In [25]:
cleanTrainData = pd.get_dummies(cleanTrainData,
                     columns = categorical_columns,
                     dtype=float
                     )

cleanTrainData

,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,barrio_legal,...,iluminacion_Malo,iluminacion_Regular,calidad_acabados_cocina_Integral,calidad_acabados_cocina_Semi-Integral,calidad_acabados_cocina_Sencillo,calidad_acabados_cocina_Sin Acabados,tipo_garaje_Comunal,tipo_garaje_Exclusivo,tipo_garaje_No Tiene,tipo_garaje_Privado
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,3.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
11,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18327,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
18328,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
18329,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
